In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1K3QJmxvgc0_ZwXawSeV_oQll-6WeibaO", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/01_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# Tokenization and Byte Pair Encoding -- Vizuara

> **What you will build:** A complete BPE tokenizer from scratch, train it on real text, and compare it against GPT-2's production tokenizer.

## 1. Why Does This Matter?

Neural networks operate on numbers, not text. Before a language model can process a single word, every character of input must be converted into a numerical representation. This conversion -- **tokenization** -- is the very first step in the entire training pipeline.

Get it wrong, and nothing downstream works. A broken tokenizer means the model receives garbage input, and no amount of clever architecture or optimization can fix that.

In this notebook, we will build a Byte Pair Encoding (BPE) tokenizer from scratch, understand why it dominates modern NLP, and see exactly how GPT-2 and LLaMA handle text.

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

There are three approaches to tokenization, and understanding the tradeoffs between them is essential.

**Character-level:** Every character gets a unique ID. Tiny vocabulary (~256), but sequences become extremely long.

**Word-level:** Every word gets a unique ID. Short sequences, but the vocabulary explodes to millions. Unknown words simply cannot be processed.

**Subword (BPE):** The sweet spot. Common words stay as single tokens. Rare words get split into meaningful pieces.

Let us see this concretely.

In [ ]:
#@title 🎧 Code Walkthrough: Char Word Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_04_char_word_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Character-level tokenization
text = "transformer"
char_tokens = [ord(c) for c in text]
print(f"Character-level: '{text}' -> {char_tokens}")
print(f"  Vocabulary size: 256 (ASCII)")
print(f"  Sequence length: {len(char_tokens)} tokens for one word!\n")

# Word-level tokenization
vocabulary = {"the": 0, "cat": 1, "sat": 2, "on": 3, "mat": 4}
sentence = "the cat sat on the mat"
word_tokens = [vocabulary[w] for w in sentence.split()]
print(f"Word-level: '{sentence}' -> {word_tokens}")
print(f"  Vocabulary size: {len(vocabulary)} (just for this sentence)")
print(f"  Sequence length: {len(word_tokens)} tokens")
print(f"  Problem: What about 'transformer'? -> KeyError!")

In [ ]:
#@title 🎧 Listen: Visualize Tradeoff Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_05_visualize_tradeoff_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us visualize the tradeoff between vocabulary size and sequence length.

In [ ]:
#@title 🎧 What to Look For: Visualize Tradeoff Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_06_visualize_tradeoff_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Simulated data for different tokenization strategies
strategies = ['Character\n(~256 tokens)', 'BPE\n(~50K tokens)', 'Word\n(~170K+ tokens)']
vocab_sizes = [256, 50000, 170000]
seq_lengths = [11, 3, 1]  # Tokens for "transformer"
colors = ['#e74c3c', '#2ecc71', '#3498db']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.bar(strategies, vocab_sizes, color=colors, edgecolor='black', linewidth=0.5)
ax1.set_ylabel('Vocabulary Size', fontsize=12)
ax1.set_title('Vocabulary Size by Strategy', fontsize=13, fontweight='bold')
ax1.set_yscale('log')
ax1.grid(axis='y', alpha=0.3)

ax2.bar(strategies, seq_lengths, color=colors, edgecolor='black', linewidth=0.5)
ax2.set_ylabel('Tokens per Word', fontsize=12)
ax2.set_title('Sequence Length (for "transformer")', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('tokenization_tradeoff.png', dpi=150, bbox_inches='tight')
plt.show()
print("Checkpoint: BPE finds the sweet spot between vocabulary size and sequence length.")

In [ ]:
#@title 🎧 Listen: Mathematics
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_07_mathematics.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

BPE is built on a simple greedy algorithm. Starting from individual characters, we repeatedly merge the most frequent adjacent pair of tokens.

Given a corpus $C$ with word frequencies $f(w)$, BPE proceeds as:

**Step 1:** Initialize vocabulary $V$ with all individual characters.

**Step 2:** For each word $w$, split into character sequence: $w = c_1, c_2, \ldots, c_n$

**Step 3:** Count all adjacent pairs across the corpus:

$$\text{count}(a, b) = \sum_{w \in C} f(w) \cdot \text{occurrences of } (a, b) \text{ in } w$$

**Step 4:** Merge the pair $(a^*, b^*)$ with the highest count:

$$(a^*, b^*) = \arg\max_{(a,b)} \text{count}(a, b)$$

**Step 5:** Replace all occurrences of $(a^*, b^*)$ with the new token $ab$ in every word.

**Step 6:** Add $ab$ to $V$. Repeat from Step 3 until $|V|$ reaches the desired size.

The **embedding memory** cost for vocabulary of size $V$ with embedding dimension $d$ is:

$$\text{Memory} = V \times d \times 4 \text{ bytes (for float32)}$$

For GPT-2 ($V = 50{,}257$, $d = 768$): $\text{Memory} = 50{,}257 \times 768 \times 4 \approx 154$ MB.

In [ ]:
#@title 🎧 Code Walkthrough: Embedding Memory Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_08_embedding_memory_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Let us compute embedding memory for different vocabulary sizes
d = 4096  # embedding dimension (common for large LLMs)
bytes_per_float = 4  # float32

print("Embedding Table Memory (d=4096, float32):")
print("-" * 45)
for V in [256, 32000, 50257, 100000]:
    mem_bytes = V * d * bytes_per_float
    mem_mb = mem_bytes / (1024 ** 2)
    print(f"  V = {V:>7,d}  ->  {mem_mb:>8.1f} MB")

print("\nThis is why vocabulary size is an engineering tradeoff!")

In [ ]:
#@title 🎧 Transition: Build Components Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_09_build_components_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let's Build It -- Component by Component

### 4.1 Word Frequency Counter

First, we need to count how often each word appears in our corpus, represented as sequences of characters with an end-of-word marker.

In [ ]:
#@title 🎧 Code Walkthrough: Word Freq Counter Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_10_word_freq_counter_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
import re
from collections import Counter, defaultdict

def get_word_frequencies(text):
    """
    Count word frequencies in the corpus.
    Each word is represented as a tuple of characters + end-of-word marker '_'.
    """
    words = text.lower().split()
    word_freqs = Counter(words)

    # Convert to character sequences with end-of-word marker
    vocab_freqs = {}
    for word, freq in word_freqs.items():
        # Split word into characters, add end marker
        char_tuple = tuple(list(word) + ['_'])
        vocab_freqs[char_tuple] = freq

    return vocab_freqs

# Test on a small corpus
corpus = "low low low low low lowest lowest newer newer newer wider wider wider wider"
freqs = get_word_frequencies(corpus)

print("Word frequencies (as character tuples):")
for word, freq in sorted(freqs.items(), key=lambda x: -x[1]):
    print(f"  {' '.join(word):20s}  (freq: {freq})")

In [ ]:
#@title 🎧 Listen: Pair Counter Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_11_pair_counter_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Pair Counter

Next, we count all adjacent pairs of tokens across the corpus.

In [ ]:
#@title 🎧 Code Walkthrough: Pair Counter Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_12_pair_counter_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def get_pair_counts(vocab_freqs):
    """Count frequencies of all adjacent token pairs in the vocabulary."""
    pair_counts = defaultdict(int)

    for word, freq in vocab_freqs.items():
        # Get all adjacent pairs in this word
        for i in range(len(word) - 1):
            pair = (word[i], word[i + 1])
            pair_counts[pair] += freq

    return pair_counts

pairs = get_pair_counts(freqs)
print("Top 10 adjacent pairs:")
for pair, count in sorted(pairs.items(), key=lambda x: -x[1])[:10]:
    print(f"  ({pair[0]:>3s}, {pair[1]:<3s})  ->  {count} times")

In [ ]:
#@title 🎧 Listen: Merge Op Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_13_merge_op_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Merge Operation

Now the core operation: merging the most frequent pair into a new token.

In [ ]:
#@title 🎧 Code Walkthrough: Merge Op Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_14_merge_op_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def merge_pair(vocab_freqs, pair_to_merge):
    """
    Merge all occurrences of pair_to_merge in the vocabulary.
    Returns a new vocabulary with the merged token.
    """
    new_vocab = {}
    bigram = pair_to_merge

    for word, freq in vocab_freqs.items():
        new_word = []
        i = 0
        while i < len(word):
            # Check if current position matches the pair to merge
            if i < len(word) - 1 and word[i] == bigram[0] and word[i + 1] == bigram[1]:
                # Merge the pair into a single token
                new_word.append(bigram[0] + bigram[1])
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_vocab[tuple(new_word)] = freq

    return new_vocab

# Test: merge the most frequent pair
best_pair = max(pairs, key=pairs.get)
print(f"Most frequent pair: {best_pair} (count: {pairs[best_pair]})")
print(f"\nBefore merge:")
for word, freq in sorted(freqs.items(), key=lambda x: -x[1]):
    print(f"  {' '.join(word):20s}  (freq: {freq})")

freqs_after = merge_pair(freqs, best_pair)
print(f"\nAfter merging {best_pair}:")
for word, freq in sorted(freqs_after.items(), key=lambda x: -x[1]):
    print(f"  {' '.join(word):20s}  (freq: {freq})")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_15_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn: Complete the BPE Trainer

### TODO 1: Implement the full BPE training loop

Complete the `train_bpe` function that runs the merge process for `num_merges` iterations.

In [ ]:
#@title 🎧 Before You Start: Todo1 Completion
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_16_todo1_completion.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def train_bpe(text, num_merges):
    """
    Train a BPE tokenizer on the given text.

    Args:
        text: Raw text corpus
        num_merges: Number of merge operations to perform

    Returns:
        merge_rules: List of (pair, merged_token) tuples
        final_vocab: The vocabulary after all merges
    """
    vocab_freqs = get_word_frequencies(text)
    merge_rules = []

    for step in range(num_merges):
        # TODO: Count all adjacent pairs
        pair_counts = ...  # YOUR CODE HERE

        # TODO: Find the most frequent pair
        if not pair_counts:
            break
        best_pair = ...  # YOUR CODE HERE

        # TODO: Merge the best pair in the vocabulary
        vocab_freqs = ...  # YOUR CODE HERE

        # Record the merge rule
        merged_token = best_pair[0] + best_pair[1]
        merge_rules.append((best_pair, merged_token))

        if step < 5 or step == num_merges - 1:
            print(f"Step {step+1}: Merge {best_pair} -> '{merged_token}' (count: {pair_counts[best_pair]})")

    return merge_rules, vocab_freqs

# SOLUTION (uncomment to verify):
# def train_bpe(text, num_merges):
#     vocab_freqs = get_word_frequencies(text)
#     merge_rules = []
#     for step in range(num_merges):
#         pair_counts = get_pair_counts(vocab_freqs)
#         if not pair_counts:
#             break
#         best_pair = max(pair_counts, key=pair_counts.get)
#         vocab_freqs = merge_pair(vocab_freqs, best_pair)
#         merged_token = best_pair[0] + best_pair[1]
#         merge_rules.append((best_pair, merged_token))
#         if step < 5 or step == num_merges - 1:
#             print(f"Step {step+1}: Merge {best_pair} -> '{merged_token}' (count: {pair_counts[best_pair]})")
#     return merge_rules, vocab_freqs

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_17_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Implement the BPE Encoder

Now implement the `encode` function that applies learned merge rules to tokenize new text.

In [ ]:
#@title 🎧 Before You Start: Todo2 Completion
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_18_todo2_completion.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def encode_bpe(text, merge_rules):
    """
    Encode text using learned BPE merge rules.

    Args:
        text: Input string to tokenize
        merge_rules: List of (pair, merged_token) from training

    Returns:
        List of BPE tokens
    """
    words = text.lower().split()
    all_tokens = []

    for word in words:
        # Start with character-level tokens + end marker
        tokens = list(word) + ['_']

        # TODO: Apply each merge rule in order
        for pair, merged in merge_rules:
            # YOUR CODE: scan tokens, merge matching adjacent pairs
            new_tokens = []
            i = 0
            while i < len(tokens):
                # Check if current pair matches the merge rule
                pass  # YOUR CODE HERE
                i += 1
            tokens = new_tokens

        all_tokens.extend(tokens)

    return all_tokens

# SOLUTION (uncomment to verify):
# def encode_bpe(text, merge_rules):
#     words = text.lower().split()
#     all_tokens = []
#     for word in words:
#         tokens = list(word) + ['_']
#         for pair, merged in merge_rules:
#             new_tokens = []
#             i = 0
#             while i < len(tokens):
#                 if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i+1] == pair[1]:
#                     new_tokens.append(merged)
#                     i += 2
#                 else:
#                     new_tokens.append(tokens[i])
#                     i += 1
#             tokens = new_tokens
#         all_tokens.extend(tokens)
#     return all_tokens

In [ ]:
#@title 🎧 Transition: Putting It Together Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_19_putting_it_together_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

Let us train our BPE tokenizer on a real corpus and see it in action.

In [ ]:
#@title 🎧 Code Walkthrough: Train On Corpus Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_20_train_on_corpus_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Train on our example corpus
corpus = "low low low low low lowest lowest newer newer newer wider wider wider wider"
merge_rules, final_vocab = train_bpe(corpus, num_merges=10)

print("\n" + "="*50)
print("Final vocabulary after training:")
print("="*50)
for word, freq in sorted(final_vocab.items(), key=lambda x: -x[1]):
    print(f"  {' '.join(word):20s}  (freq: {freq})")

print(f"\nMerge rules learned: {len(merge_rules)}")
for pair, merged in merge_rules:
    print(f"  {pair} -> '{merged}'")

In [ ]:
#@title 🎧 Listen: Encode Text Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_21_encode_text_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let us encode some text and see how BPE handles known and unknown words.

In [ ]:
#@title 🎧 Code Walkthrough: Encoding Examples Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_22_encoding_examples_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Encode text using our trained tokenizer
test_texts = [
    "low",           # Common word -- should be single token
    "lowest",        # Known compound -- should split meaningfully
    "newer",         # Known word
    "widest",        # UNSEEN word -- can BPE handle it?
    "lower",         # UNSEEN word -- interesting test
]

print("BPE Encoding Results:")
print("-" * 50)
for text in test_texts:
    tokens = encode_bpe(text, merge_rules)
    print(f"  '{text:10s}'  ->  {tokens}")

print("\nNotice: BPE can handle unseen words by breaking them")
print("into known subword pieces!")

In [ ]:
#@title 🎧 Transition: Comparing Gpt2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_23_comparing_gpt2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Comparing with GPT-2's Tokenizer

Let us compare our toy BPE with the production tokenizer used by GPT-2.

In [ ]:
#@title 🎧 Code Walkthrough: Tiktoken Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_24_tiktoken_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
import subprocess
subprocess.check_call(['pip', 'install', '-q', 'tiktoken'])

import tiktoken

enc = tiktoken.get_encoding("gpt2")

test_sentences = [
    "The cat sat on the mat",
    "Transformers are incredible",
    "supercalifragilisticexpialidocious",
    "def hello_world():",
    "GPT-4 uses BPE tokenization",
]

print("GPT-2 Tokenizer (tiktoken):")
print("=" * 60)
for sentence in test_sentences:
    token_ids = enc.encode(sentence)
    token_strings = [enc.decode([t]) for t in token_ids]
    print(f"\n  Input:  '{sentence}'")
    print(f"  IDs:    {token_ids}")
    print(f"  Tokens: {token_strings}")
    print(f"  Count:  {len(token_ids)} tokens")

print(f"\nGPT-2 vocabulary size: {enc.n_vocab:,}")

In [ ]:
#@title 🎧 Listen: Token Dist Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_25_token_dist_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Visualization: Token distribution

In [ ]:
#@title 🎧 What to Look For: Token Dist Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_26_token_dist_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize how GPT-2 tokenizes different types of text
import matplotlib.pyplot as plt

text_types = {
    'English': "The quick brown fox jumps over the lazy dog",
    'Code': "def fibonacci(n): return n if n <= 1 else fibonacci(n-1) + fibonacci(n-2)",
    'Math': "The eigenvalue decomposition A = PDP^{-1} yields orthogonal eigenvectors",
    'Names': "Schwarzenegger and Dostoyevsky met at Constantinople",
}

fig, ax = plt.subplots(figsize=(10, 5))
categories = []
token_counts = []
chars_per_token = []

for name, text in text_types.items():
    tokens = enc.encode(text)
    categories.append(name)
    token_counts.append(len(tokens))
    chars_per_token.append(len(text) / len(tokens))

x = np.arange(len(categories))
width = 0.35

bars1 = ax.bar(x - width/2, token_counts, width, label='Token Count', color='#3498db', edgecolor='black', linewidth=0.5)
bars2 = ax.bar(x + width/2, chars_per_token, width, label='Chars/Token', color='#e74c3c', edgecolor='black', linewidth=0.5)

ax.set_ylabel('Count', fontsize=12)
ax.set_title('GPT-2 Tokenization Efficiency by Text Type', fontsize=13, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=11)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('gpt2_tokenization.png', dpi=150, bbox_inches='tight')
plt.show()
print("Checkpoint: Common English gets ~4 chars/token; rare names get fewer.")

In [ ]:
#@title 🎧 Transition: Final Output Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_27_final_output_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

Let us build a complete, self-contained BPE tokenizer class and demonstrate it end-to-end.

In [ ]:
#@title 🎧 Narration: Simplebpe Class Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_28_simplebpe_class_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class SimpleBPE:
    """A complete BPE tokenizer built from scratch."""

    def __init__(self):
        self.merge_rules = []
        self.vocab = set()

    def train(self, text, num_merges=100):
        """Train the BPE tokenizer on a text corpus."""
        vocab_freqs = get_word_frequencies(text)
        self.merge_rules = []

        for step in range(num_merges):
            pair_counts = get_pair_counts(vocab_freqs)
            if not pair_counts:
                break
            best_pair = max(pair_counts, key=pair_counts.get)
            vocab_freqs = merge_pair(vocab_freqs, best_pair)
            merged_token = best_pair[0] + best_pair[1]
            self.merge_rules.append((best_pair, merged_token))

        # Build final vocabulary
        self.vocab = set()
        for word in vocab_freqs:
            for token in word:
                self.vocab.add(token)

        print(f"Training complete: {len(self.merge_rules)} merges, {len(self.vocab)} unique tokens")

    def encode(self, text):
        """Encode text into BPE tokens."""
        return encode_bpe(text, self.merge_rules)

    def vocab_size(self):
        return len(self.vocab)

# Train on a larger corpus
large_corpus = """
The transformer architecture has revolutionized natural language processing.
Transformers use self-attention mechanisms to process sequences in parallel.
The key innovation is the attention mechanism which computes query key value products.
Language models predict the next token given previous tokens in the sequence.
Training requires tokenization data loading and optimization.
The training pipeline converts raw text into trained model weights.
Byte pair encoding finds the optimal subword vocabulary for the corpus.
""" * 10  # Repeat for more training data

tokenizer = SimpleBPE()
tokenizer.train(large_corpus, num_merges=50)

print(f"\nVocabulary size: {tokenizer.vocab_size()}")
print(f"\nEncoding examples:")
for test in ["transformer", "attention", "tokenization", "neural network"]:
    tokens = tokenizer.encode(test)
    print(f"  '{test}' -> {tokens}")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_29_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

**What we built:** A complete BPE tokenizer from scratch -- the same algorithm used in GPT-2, GPT-4, and LLaMA.

**Key takeaways:**
- Tokenization is the critical first step -- get it wrong and nothing downstream works
- BPE finds the sweet spot between character-level and word-level approaches
- The algorithm is greedy: repeatedly merge the most frequent adjacent pair
- Vocabulary size is a direct engineering tradeoff: larger means more memory for embeddings, smaller means longer sequences

**What is next:** In Notebook 2, we will take these token sequences and build the dataset pipeline -- creating input-target pairs for next-token prediction using a sliding window approach.